In [ ]:
from datetime import datetime
from dateutil.relativedelta import relativedelta
import json
import os
from typing import Dict, List

import boto3
import requests

def get_taxi_data(formatted_datetime: str) -> List[Dict]:
    """
    Retrieves taxi data for the given date.
    
    Parameters:
        formatted_datetime (str): The date in "YYYY-MM-DD" format.
    
    Returns:
        List[Dict]: A list of dictionaries contatining the taxi_data as a JSON.
    """
    
    taxi_url = f"https://data.cityofchicago.org/resource/ajtu-isnz.json?$where=trip_start_timestamp >= '{formatted_datetime}T00:00:00' AND trip_start_timestamp <= '{formatted_datetime}T23:59:59'&$limit=30000"
    headers = {"X-App-Token": os.environ.get("CHICAGO_KEY")}

    response_taxi = requests.get(taxi_url)
    taxi_data = response_taxi.json()
    
    return taxi_data
    
def get_weather_data(formatted_datetime: str) -> Dict:
    """
    Retrieve weather data from the Open Meteo API for a specific date and location.

    Parameters:
        formatted_datetime (str): The date in "YYYY-MM-DD" format.

    Returns:
        Dict: A dictionary containing weather data, including temperature at 2 meters, wind speed at 10 meters,
            rain, and precipitation for the specified date and location.
    """
    
    if not isinstance(formatted_datetime, str):
        raise TypeError("Value must be a string!")
    
    weather_url = "https://archive-api.open-meteo.com/v1/era5"
    
    params = {
        "latitude": 41.85,
        "longitude": -87.65,
        "start_date": formatted_datetime,
        "end_date": formatted_datetime,
        "hourly": "temperature_2m,wind_speed_10m,rain,precipitation"
    }
    
    response_weather = requests.get(weather_url, params=params)
    weather_data = response_weather.json()
    
    return weather_data
    
def upload_to_s3(data: List, folder_name: str, filename: str) -> None:
    """
    Upload data to an Amazon S3 bucket.

    Parameters:
        data (List): A list of dictionaries containing the data to be uploaded, either taxi or weather data.
        folder_name (str): The name of the folder within the S3 bucket where the data will be stored.
        filename (str): The name of the file to be created within the specified folder.

    Returns:
        None: This function does not return anything.
    """
    client = boto3.client("s3")
    client.put_object(
        Bucket="cubix-chicago-taxi-bb",
        Key=f"raw_data/to_processed/{folder_name}/{filename}",
        Body=json.dumps(data)
    )

    
def lambda_handler(event, context):
    current_datetime = datetime.now() - relativedelta(months=2)
    formatted_datetime = current_datetime.strftime("%Y-%m-%d")
    
    taxi_data_api_call = get_taxi_data(formatted_datetime)
    weather_data_api_call = get_weather_data(formatted_datetime)
    
    taxi_filename = f"taxi_raw_{formatted_datetime}.json"
    weather_filename = f"weather_raw_{formatted_datetime}.json"
    
    upload_to_s3(data=taxi_data_api_call, filename=taxi_filename, folder_name="taxi_data")
    print("Taxi data has been uploaded!")

    upload_to_s3(data=weather_data_api_call, filename=weather_filename, folder_name="weather_data")
    print("Weather data has been uploaded!")
